<a href="https://colab.research.google.com/github/izu-hiro/RAG-Pageindex/blob/main/prototipo_rag/quary.teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Definindo a LLM

In [1]:
from openai import AsyncOpenAI
from getpass import getpass

openrouterKey = getpass("Insira a sua chave API: ")

async def call_llm(prompt, model="deepseek/deepseek-chat-v3.1:free", temperature=0):
    client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1", api_key=openrouterKey)
    response = await client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

Conectando com o PageIndex

In [2]:
from pageindex import PageIndexClient

pi_client = PageIndexClient(api_key=getpass("Insira a chave API do Page Index: "))

Contexto utilizado na geração da resposta

In [13]:
from pypdf import PdfWriter
from os import listdir
from os.path import isfile, join

documentos = ['documentos/'+f for f in listdir('documentos') if isfile(join('documentos', f))]

merger = PdfWriter()
for documento in documentos:
    merger.append(documento)
merger.write("documentoUnido.pdf")
merger.close()



Enviar arquivo e pegar o ID

In [14]:
result = pi_client.submit_document("./documentoUnido.pdf")
doc_id = result["doc_id"]

# Get OCR results in page format (default)
ocr_result = pi_client.get_ocr(doc_id)
if ocr_result.get("status") == "completed":
    print("OCR Results:", ocr_result.get("result"))

'''
# Get OCR results in node format
ocr_result = pi_client.get_ocr(doc_id, format="node")
if ocr_result.get("status") == "completed":
    print("OCR Results:", ocr_result.get("result"))

# Get OCR results in raw format (concatenated markdown)
ocr_result = pi_client.get_ocr(doc_id, format="raw")
if ocr_result.get("status") == "completed":
    print("Raw Markdown:", ocr_result.get("result"))
'''

tree_result = pi_client.get_tree(doc_id)
if tree_result.get("status") == "completed":
    print("PageIndex Tree Structure:", tree_result.get("result"))

Retrieval

In [15]:
from time import sleep
def retrieval(query):
    while True:
        if pi_client.is_retrieval_ready(doc_id):
            retrieval = pi_client.submit_query(doc_id, query)
            retrieval_id = retrieval['retrieval_id']
            break
        else:
            print("Document is not ready for retrieval yet")
            sleep(5)

    while True:
        retrieval_result = pi_client.get_retrieval(retrieval_id)
        if retrieval_result.get("status") == "completed":
            return retrieval_result.get('result')
        sleep(1)


Geração da resposta

In [16]:
perguntas = [
    "Qual é a idade mínima para poder ingressar na fatec?",
    "Quanto custa pra estudar numa instituicao do centro paula souza?",
    "Como q funciona o processo de inscrição no vestibulinho da etec?",
    "Fatec tem programa de mestrado? se sim, qnd começam as aulas do mestrado de 2025?"
]

for query in perguntas:
    relevant_content = retrieval(query)
    print(relevant_content)
    answer_prompt = f"""
Responda à pergunta com base apenas no contexto fornecido abaixo.

- Interprete a pergunta em **português**, mesmo que ela contenha:
  - Gírias ou linguagem informal
  - Erros de digitação ou ortografia
  - Abreviações ou escrita de forma não convencional
  - Estruturas de frase incompletas ou confusas
- Se houver ambiguidade, faça a melhor interpretação possível considerando o contexto.
- Mantenha a resposta **clara, completa e em português correto**.
- Não invente informações que não estejam no contexto.
- Use exemplos ou explicações adicionais, se necessário, para garantir que a resposta seja compreendida.


    Pergunta do usuário: {query}
    Contexto: {relevant_content}

    Resposta:
    """

    answer = await call_llm(answer_prompt)

    print("Pergunta original:\n", query)
    print("\nResposta gerada (PT):\n", answer)


Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet
Document is not ready for retrieval yet


NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy (Free model publication). Configure: https://openrouter.ai/settings/privacy', 'code': 404}}